In [21]:
!pip install python-pptx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import os
_colab_dir = '/content/drive/MyDrive/Colab Notebooks/post_per100k'
os.chdir(_colab_dir)

In [23]:
import pandas as pd
KTQurl = 'https://www.city.kitakyushu.lg.jp/ho-huku/18901209.html'
KTQ_per100K7 = pd.read_html(KTQurl, header = 0)[1].iat[1,1]

In [24]:
NHKurl = 'https://raw.githubusercontent.com/u-10bei/covid-19_JPdata/main/data/COVID-19_NHK.csv'
NHK = pd.read_csv(NHKurl).iloc[:,[0,4]]
NHK.reset_index(inplace=True)
NHK['Id'] = NHK['index']+1
NHK['都道府県'] = NHK['Id'].apply(lambda _: str(_))+'.'+NHK['Pref']
NHK['人'] = NHK['per100K7'].round(2)
NHK_chart = NHK.iloc[0:20,[4,5]]

In [25]:
print(NHK_chart)
print(NHK_chart.dtypes)

       都道府県        人
0     1.徳島県  1703.68
1     2.高知県  1478.90
2     3.長崎県  1469.23
3    4.鹿児島県  1458.08
4     5.宮崎県  1375.22
5     6.佐賀県  1260.60
6     7.沖縄県  1248.40
7     8.広島県  1217.88
8     9.香川県  1207.37
9    10.福岡県  1177.32
10   11.熊本県  1147.61
11   12.石川県  1137.63
12   13.岡山県  1124.74
13   14.青森県  1102.60
14   15.富山県  1098.17
15   16.三重県  1095.15
16  17.和歌山県  1092.48
17   18.奈良県  1087.45
18   19.滋賀県  1074.62
19   20.大阪府  1070.56
都道府県     object
人       float64
dtype: object


In [26]:
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.util import Pt 

In [27]:
from datetime import date, timedelta
yesterday = date.today() - timedelta(days=1)
reiwa = 'R'+str(yesterday.year%100-18).zfill(2)
reiwa2 = '令和'+str(yesterday.year%100-18)+'年'
wareki = reiwa + yesterday.strftime('%m%d')
wareki2 = reiwa2 + str(yesterday.month) + '月' + str(yesterday.day) + '日現在'

In [28]:
ppturl = 'hinagata.pptx'
prs = Presentation(ppturl)
sld = prs.slides[0]

In [29]:
text1 = sld.shapes[3].text_frame.paragraphs[0]
text1.text = wareki2
text1.font.bold = True
text1.font.size = Pt(14)

text2 = sld.shapes[4].text_frame.paragraphs[0]
text2.text = '※北九州市　'+KTQ_per100K7
text2.font.size = Pt(16)

chart = sld.shapes[0].chart
chart_data = CategoryChartData()
chart_data.categories = NHK_chart['都道府県']
chart_data.add_series('人',(NHK_chart['人']),'#,##0.00')
chart.replace_data(chart_data)

In [30]:
outurl = '06_' + wareki + '_感染者数（10万人あたり）.pptx'
prs.save(outurl)

In [47]:
msg_title = '１０万人あたり感染者数の資料について'
msg_text = 'おはようございます。\n表題の件について、添付ファイルのとおり送付いたします。'

In [48]:
import smtplib
from util import credential as cred
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.utils import formatdate
#SMTPのオブジェクト作成。GmailのSMTPポートは587
smtpobj = smtplib.SMTP('smtp.gmail.com', 587)

#メールサーバに対する応答
smtpobj.ehlo()
#暗号化通信開始
smtpobj.starttls()
smtpobj.ehlo()
#ログイン
smtpobj.login(cred.GOOGLE_ACCOUNT, cred.GOOGLE_PASSWORD)

(235, b'2.7.0 Accepted')

In [49]:
send_list = cred.MAIL_TO.split(',')
msg = MIMEMultipart()
msg['Subject'] = msg_title
msg['From'] = cred.GOOGLE_ACCOUNT
msg['To'] = cred.MAIL_TO
msg['Date'] = formatdate(localtime=True)
msg.attach(MIMEText(msg_text))

with open(outurl,'rb') as f:
  attachment = MIMEApplication(f.read())

attachment.add_header('Content-Disposition','attachment',filename=outurl)
msg.attach(attachment)

In [50]:
smtpobj.sendmail(cred.GOOGLE_ACCOUNT,send_list,msg.as_string())
smtpobj.quit()

(221,
 b'2.0.0 closing connection b83-20020a1f1b56000000b0038cdb2fbeb1sm2499571vkb.23 - gsmtp')